# Обучение модели на других данных и валидация их на наших

In [2]:
from ultralytics import YOLO
import os
import cv2
import shutil
import numpy as np

In [ ]:
train_images = 'soiling_dataset/train/rgbImages'  # Путь к вашему датасету с изображениями
train_mask = 'train/rgbLabels'  # Путь к вашему датасету с масками

val_images = 'cv_open_dataset/open_img'
val_mask = 'cv_open_dataset/open_msk'

OUTPUT_DIR = './datasets/train_new_data'  # Путь к выходной директории

In [3]:
os.makedirs(os.path.join(OUTPUT_DIR, 'images/train'), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, 'images/val'), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, 'labels/train'), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, 'labels/val'), exist_ok=True)

In [4]:
def convert_mask_to_yolo(mask_path, out):
    # Открываем изображение
    image = cv2.imread(mask_path)

    if image is None:
        print(f"Не удалось открыть изображение: {mask_path}")
        return
    
    height, width = image.shape[:2]

    # Создаем маску для черного цвета
    black_mask = cv2.inRange(image, (0, 0, 0), (50, 50, 50))

    # Создаем новое изображение, где черный цвет остается, а остальные цвета становятся белыми
    new_image = np.ones_like(image) * 255  # Начинаем с белого изображения
    new_image[black_mask > 0] = [0, 0, 0]  # Заменяем черные пиксели

    # Преобразуем в градации серого для нахождения контуров
    gray_image = cv2.cvtColor(new_image, cv2.COLOR_BGR2GRAY)

    # Находим контуры
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Сохраняем контуры в текстовом формате
    output_file_name = os.path.splitext(os.path.basename(mask_path))[0] + '.txt'
    output_file_path = os.path.join(OUTPUT_DIR, out, output_file_name)

    with open(output_file_path, 'w') as f:
        for index, contour in enumerate(contours):
            # Получаем координаты всех точек контура
            contour_points = contour.reshape(-1, 2)
            # Нормализуем координаты
            normalized_points = [(x / width, y / height) for x, y in contour_points]
            points_str = ' '.join(f"{x:.3f} {y:.3f}" for x, y in normalized_points)
            f.write(f"0 {points_str}\n")

In [7]:
# Копирование изображений и масок в соответствующие папки
for img in os.listdir(train_images):
    shutil.copy(os.path.join(train_images, img), os.path.join(OUTPUT_DIR, 'images/train', img))
    mask_name = img.replace('.jpg', '.png')
    convert_mask_to_yolo(os.path.join(train_mask, mask_name), 'labels/train')

In [8]:
for img in os.listdir(val_images):
    shutil.copy(os.path.join(val_images, img), os.path.join(OUTPUT_DIR, 'images/val', img))
    mask_name = img.replace('.jpg', '.png')
    convert_mask_to_yolo(os.path.join(val_mask, mask_name), 'labels/val')

In [9]:
data_yaml_content = f"""
train: train_new_data/images/train
val: train_new_data/images/val

nc: 1  # Обновите количество классов (1 для загрязнения)
names: ['contaminated']  # Обновите названия классов
"""

with open('new_data.yaml', 'w') as f:
    f.write(data_yaml_content)

print("Датасет успешно разбит и сохранен в структуре проекта.")

Датасет успешно разбит и сохранен в структуре проекта.


In [3]:
model = YOLO("yolo11n-seg.pt")

In [4]:
train_results = model.train(
    data="./new_data.yaml",  # path to dataset YAML
    epochs=150,  # number of training epochs
    imgsz=640,  # training image size
    device="0",  # device to run on, i.e. device=0 or device=0,1,2,3 or device=cpu
    batch = 7,
)

New https://pypi.org/project/ultralytics/8.3.47 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.44  Python-3.10.0 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 3050 Ti Laptop GPU, 4096MiB)
engine\trainer: task=segment, mode=train, model=yolo11n-seg.pt, data=./new_data.yaml, epochs=150, time=None, patience=100, batch=7, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train7, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=Fa

train: Scanning C:\Users\nadyl\Documents\GitHub\nornnickel_hac\datasets\train_new_data\labels\train.cache... 4000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4000/4000 [00:00<?, ?it/s]
val: Scanning C:\Users\nadyl\Documents\GitHub\nornnickel_hac\datasets\train_new_data\labels\val.cache... 250 images, 163 backgrounds, 0 corrupt: 100%|██████████| 250/250 [00:00<?, ?it/s]


Plotting labels to c:\Users\nadyl\Documents\GitHub\nornnickel_hac\runs\segment\train7\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 90 weight(decay=0.0), 101 weight(decay=0.0004921875), 100 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to c:\Users\nadyl\Documents\GitHub\nornnickel_hac\runs\segment\train7
Starting training for 150 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


      1/150      1.46G     0.7444      2.068      1.224      1.234         10        640: 100%|██████████| 572/572 [01:22<00:00,  6.97it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:02<00:00,  7.52it/s]

                   all        250        202       0.33      0.401      0.199      0.132      0.307      0.366      0.166      0.101


...

    143/150      1.33G     0.3317     0.7241     0.2066     0.9332          4        640: 100%|██████████| 572/572 [01:35<00:00,  6.00it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:02<00:00,  6.94it/s]

                   all        250        202      0.293      0.337      0.157     0.0818      0.314      0.282      0.145      0.072



      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


    144/150      1.33G     0.3311     0.7163     0.2007     0.9264          7        640:  35%|███▍      | 198/572 [00:29<00:55,  6.80it/s]


KeyboardInterrupt: 

Ход обучения показал что особо сильно оно не улучшилось, поэтому смысла в обучении нет.